In [ ]:
import json
from IPython.display import JSON

from unity_sds_client.unity import Unity
from unity_sds_client.unity import UnityEnvironments
from unity_sds_client.unity_session import UnitySession
from unity_sds_client.unity_services import UnityServices as services
from unity_sds_client.resources.collection import Collection

In [ ]:
# We will set the environment to 'DEV' here but this should be set to test or prod eventually.
s = Unity(UnityEnvironments.TEST)
# set the venue for interacting with venue specific services
# if your venue id is a single string, use the following

In [ ]:
dataManager = s.client(services.DATA_SERVICE)
collection_id = "urn:nasa:unity:unity:test:SBG-L2B_FRCOV___1"
cd = dataManager.get_collection_data(Collection(collection_id))
for dataset in cd:
    print(dataset.data_create_time)
    print(f'dataset name: {dataset.data_begin_time}')
    print(f'dataset name: {dataset.id}' )
    print(f'dataset soil props: ' + str(dataset.properties['soil']) )
    for f in dataset.datafiles:
        print(f)
        #print("	" + f.location + ", roles: " + str(f.roles) + ", type: " + f.type + ", description: " + f.description + ", title: " + f.title)

In [ ]:
# Check the metadata for soil cover
key = "soil"
threshold = '0.5'
pixel_count = 100

for dataset in cd:
    if dataset.properties[key][threshold] > pixel_count:
        print(f'Execute l2 algorithm for {dataset.id}')

In [ ]:
# Execute the L2 workflow

In [ ]:
import requests
import getpass

username = getpass.getpass('Enter your username')
password = getpass.getpass('Enter your password')

data = {
  "conf": {
    "cwl_workflow":"https://raw.githubusercontent.com/unity-sds/sbg-workflows/main/trait-estimate/sbg-trait-estimate-workflow.cwl",
    "args_as_json":"https://raw.githubusercontent.com/unity-sds/sbg-workflows/main/trait-estimate/sbg-trait-estimate-workflow.test.yml"
  },
  "note": "cli run"
}
# Will be replaced by the WPS execute scripts
# Auth: Only needed until we get the cognito user stuff worked out,
response = requests.post("http://k8s-airflow-airflowi-5b65901547-1494561622.us-west-2.elb.amazonaws.com:5000/api/v1/dags/cwl-dag/dagRuns",auth=(username, password), headers={"accept":"application/json", "Content-Type":"application/json"}, json=data)
response.json()

In [ ]:
curl -X 'POST' \
  'http://k8s-airflow-airflowi-5b65901547-1494561622.us-west-2.elb.amazonaws.com:5000/api/v1/dags/cwl-dag/dagRuns' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
  "conf": {
    "cwl_workflow":"https://raw.githubusercontent.com/unity-sds/sbg-workflows/main/trait-estimate/sbg-trait-estimate-workflow.cwl",
    "args_as_json":"https://raw.githubusercontent.com/unity-sds/sbg-workflows/main/trait-estimate/sbg-trait-estimate-workflow.test.yml"
  },
  "note": "cli run"
}'